In [14]:
pip install pg8000

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import ssl
import pg8000
import re

In [ ]:
# conexion a RDS usando el bundle de Amazon, esto no es necesario desde lambda
RDS_CA_PATH = os.getenv(
    "RDS_SSL_CA",
    r"K:\documentos\UNGS\Proyecto Profesional I\Tp principal 2-2025\AlimentAPP\global-bundle.pem",
)

ssl_context = ssl.create_default_context(cafile=RDS_CA_PATH)

In [15]:
# Variables
ENV = 'dev'

In [ ]:
PASSWORD_INPUT = input('ingrese contraseña')
def get_connection():
    """
    Abre una conexion pg8000 con SSL.
    """
    return pg8000.connect(
        host='db-alimentapp.cm7oyoiiqchb.us-east-1.rds.amazonaws.com',
        port=5432,
        database='postgres',
        user='labo_team',
        password=PASSWORD_INPUT,
        ssl_context=ssl_context,
        timeout=10,
    )

In [ ]:
def run_query(sql: str):
    """
    Ejecuta un SELECT y devuelve filas como diccionarios.
    """
    conn = get_connection()
    try:
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        if not rows:
            return []
        columns = [desc[0] for desc in cur.description]
        return [dict(zip(columns, row)) for row in rows]
    finally:
        conn.close()

def run_command(sql: str):
    """
    Ejecuta INSERT/UPDATE/DELETE y hace commit.
    """
    conn = get_connection()
    try:
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    finally:
        conn.close()

# get_clients

In [17]:
get_clients_query = f'SELECT * FROM {ENV}.cliente'

# Obtengo todos los clientes
clients = run_query(get_clients_query)
print(clients)

[{'id': 1, 'razon_social': 'Panificados SRL', 'email': 'ventas@panificados.com', 'cuil': '30-12345678-9', 'nombre_contacto': 'Ana', 'apellido_contacto': 'Gimenez', 'telefono': '+54 11 5555-6677', 'activo': True, 'created_at': datetime.datetime(2025, 9, 27, 19, 54, 41, 680028, tzinfo=datetime.timezone.utc)}, {'id': 3, 'razon_social': 'supermercado don carlos', 'email': 'compras@doncarlos.com', 'cuil': '30-98765432-1', 'nombre_contacto': 'pedro', 'apellido_contacto': 'ramirez', 'telefono': '11-9876-5432', 'activo': True, 'created_at': datetime.datetime(2025, 9, 27, 20, 0, 23, 822441, tzinfo=datetime.timezone.utc)}, {'id': 4, 'razon_social': 'restaurante la cocina gourmet', 'email': 'pedidos@cocinagourmet.com', 'cuil': '27-87654321-0', 'nombre_contacto': 'Sofia', 'apellido_contacto': 'Morales', 'telefono': '11-6543-2109', 'activo': True, 'created_at': datetime.datetime(2025, 9, 27, 20, 0, 23, 822441, tzinfo=datetime.timezone.utc)}, {'id': 5, 'razon_social': 'distribuidora zona norte srl',

# Post nuevo cliente

In [ ]:
class ValidationError(Exception):
    pass


NAME_REGEX = re.compile(r"^[A-Za-zÁÉÍÓÚÑáéíóúñ\s']+$")
PHONE_REGEX = re.compile(r"^[0-9+\-\s()]+$")
EMAIL_REGEX = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
CUIL_REGEX = re.compile(r"^\d{2}-\d{8}-\d$")  # ajustalo si aceptás otro formato

def validate_client_payload(payload: dict[str, str]) -> dict[str, str]:
    """
    Valida que todos los campos del cliente estén presentes y en formato correcto.
    """
    required = [
        "razon_social",
        "email",
        "cuil",
        "nombre_contacto",
        "apellido_contacto",
        "telefono",
    ]
    missing = [key for key in required if not payload.get(key)]
    if missing:
        raise ValidationError(f"Faltan campos obligatorios: {', '.join(missing)}")

    email = payload["email"].strip()
    if not EMAIL_REGEX.fullmatch(email):
        raise ValidationError("email no tiene un formato válido.")

    cuil = payload["cuil"].strip()
    if not CUIL_REGEX.fullmatch(cuil):
        raise ValidationError("cuil no respeta el formato ##-########-#.")

    nombre = payload["nombre_contacto"].strip()
    if not NAME_REGEX.fullmatch(nombre):
        raise ValidationError("nombre_contacto solo admite letras y espacios.")

    apellido = payload["apellido_contacto"].strip()
    if not NAME_REGEX.fullmatch(apellido):
        raise ValidationError("apellido_contacto solo admite letras y espacios.")

    telefono = payload["telefono"].strip()
    if not PHONE_REGEX.fullmatch(telefono):
        raise ValidationError("telefono solo admite dígitos, +, -, () y espacios.")

    return {
        "razon_social": payload["razon_social"].strip(),
        "email": email,
        "cuil": cuil,
        "nombre_contacto": nombre,
        "apellido_contacto": apellido,
        "telefono": telefono,
    }

In [ ]:
input_payload = {
  "razon_social": "Panificados SRL",
  "email": "abc@hotmail.com",
  "cuil": "30-23456728-9",
  "nombre_contacto": "Ana",
  "apellido_contacto": "Gimenez",
  "telefono": "1155556677"
}

In [ ]:
try:
    data = validate_client_payload(input_payload)
except ValidationError as exc:
    #return {"statusCode": 400, "body": json.dumps({"message": str(exc)})}
    print({"statusCode": 400, "body": json.dumps({"message": str(exc)})})

In [ ]:

run_command(
    f"""
    INSERT INTO {ENV}.cliente
        (razon_social, email, cuil, nombre_contacto, apellido_contacto, telefono)
    VALUES
        ('{data["razon_social"]}', '{data["email"]}', '{data["cuil"]}',
         '{data["nombre_contacto"]}', '{data["apellido_contacto"]}', '{data["telefono"]}');
    """
)

# get orden venta

In [ ]:
input_payload = {
  "id": "3",
}

In [23]:
import json
def get_orden_venta(event, context):
    #query = f"select * from {ENV}.orden_venta limit 20;"
    query_pedidos = f"""
        SELECT 
            id as "idPedido",
            id_empleado as "idVendedor", 
            id_cliente as "idCliente",
            fecha_pedido as "fechaPedido",
            fecha_entrega_solicitada as "fechaEntrega",
            estado
        FROM {ENV}.orden_venta
        ORDER BY id
        """
        
    pedidos = run_query(query_pedidos)
        
    # Luego para cada pedido, obtenemos sus productos
    pedidos_con_productos = []
    for pedido in pedidos:
        query_productos = f"""
        SELECT 
            op.id_producto as "idProducto",
            p.nombre,
            op.cantidad
            FROM {ENV}.orden_produccion op
            LEFT JOIN {ENV}.producto p ON op.id_producto = p.id
            WHERE op.id_orden_venta = {pedido['idPedido']}
            """
            
        productos = run_query(query_productos)
            
            # Construimos el objeto del pedido con productos
        pedido_completo = {
                "idPedido": pedido['idPedido'],
                "idVendedor": pedido['idVendedor'],
                "idCliente": pedido['idCliente'],
                "productos": productos,
                "fechaPedido": pedido['fechaPedido'].isoformat() if pedido['fechaPedido'] else None,
                "fechaEntrega": pedido['fechaEntrega'].isoformat() if pedido['fechaEntrega'] else None,
                "estado": pedido['estado']
            }
            
        pedidos_con_productos.append(pedido_completo)
    return {
        "statusCode": 200,
        "body": json.dumps({"data": pedidos_con_productos}, default = str)
    }

print(get_orden_venta(None, None))

{'statusCode': 200, 'body': '{"data": [{"idPedido": 1, "idVendedor": 1, "idCliente": 1, "productos": [{"idProducto": 1, "nombre": "Empanadas de Carne x12", "cantidad": 5}, {"idProducto": 3, "nombre": "Pizza Muzzarella Grande", "cantidad": 3}, {"idProducto": 4, "nombre": "Milanesas de Pollo x6", "cantidad": 4}], "fechaPedido": "2025-09-27T21:56:40.707043+00:00", "fechaEntrega": "2025-10-01T00:00:00+00:00", "estado": "confirmada"}, {"idPedido": 2, "idVendedor": 1, "idCliente": 1, "productos": [{"idProducto": 2, "nombre": "Hamburguesas Premium x4", "cantidad": 2}, {"idProducto": 5, "nombre": "Tarta de Verdura", "cantidad": 1}], "fechaPedido": "2025-09-27T22:08:16.959086+00:00", "fechaEntrega": "2025-10-01T00:00:00+00:00", "estado": "confirmada"}, {"idPedido": 3, "idVendedor": 1, "idCliente": 1, "productos": [{"idProducto": 1, "nombre": "Empanadas de Carne x12", "cantidad": 10}, {"idProducto": 2, "nombre": "Hamburguesas Premium x4", "cantidad": 8}, {"idProducto": 3, "nombre": "Pizza Muzzar

# get orden venta by id vendedor

In [ ]:
input_payload = {
  "id": "4"
}
def get_orden_venta_by_id_vendedor(event, context):
    # Con API Gateway el body viene encapsulado dentro del event(un JSON)
    # input_payload = json.loads(event["body"])
    input_payload = event
    
    try:
        get_id_vendedor = int(input_payload["id"])
    except (ValueError, KeyError) as exc:
        return{
            "statusCode": 400,
            "body": json.dumps({"message": f"ID inválido: {exc}"}, default = str)
        }

    query = f"select * from {ENV}.orden_venta where id_empleado = {get_id_vendedor};"
    pedido = run_query(query)
    return {
        "statusCode": 200,
        "body": json.dumps({"data": pedido}, default = str)
    }

print(get_orden_venta_by_id_vendedor(input_payload, None))

# get orden venta by id cliente

In [ ]:
input_payload = {
  "id": ""
}
def get_orden_venta_by_id_cliente(event, context):
    # Con API Gateway el body viene encapsulado dentro del event(un JSON)
    # input_payload = json.loads(event["body"])
    input_payload = event
    
    try:
        get_id_cliente = int(input_payload["id"])
    except (ValueError, KeyError) as exc:
        return{
            "statusCode": 400,
            "body": json.dumps({"message": f"ID inválido: {exc}"}, default = str)
        }

    query = f"select * from {ENV}.orden_venta where id_cliente = {get_id_cliente};"
    pedido = run_query(query)
    return {
        "statusCode": 200,
        "body": json.dumps({"data": pedido}, default = str)
    }

print(get_orden_venta_by_id_cliente(input_payload, None))

# get orden venta by id pedido

In [ ]:
input_payload = {
  "id": ""
}

def get_orden_venta_by_id_pedido(event, context):
    # Con API Gateway el body viene encapsulado dentro del event(un JSON)
    # input_payload = json.loads(event["body"])
    input_payload = event
    
    try:
        get_id_pedido = int(input_payload["id"])
    except (ValueError, KeyError) as exc:
        return{
            "statusCode": 400,
            "body": json.dumps({"message": f"ID inválido: {exc}"}, default = str)
        }

    query = f"select * from {ENV}.orden_venta where id = {get_id_pedido};"
    pedido = run_query(query)
    return {
        "statusCode": 200,
        "body": json.dumps({"data": pedido}, default = str)
    }

print(get_orden_venta_by_id_pedido(input_payload, None))

# get orden venta by estado

In [ ]:
input_payload2 = {
  "estado": "pendiente",
}

def get_orden_venta_by_estado(event, context):
    input_payload = event
    
    estados_orden_venta = [
        'pendiente',
        'confirmada',
        'cancelada',
        'en_produccion',
        'lista',
        'entregada'
    ]
    
    get_estado = input_payload.get("estado")
    if not get_estado or get_estado not in estados_orden_venta:
        return {
            "statusCode": 400,
            "body": json.dumps({"message": f"El estado '{get_estado}' NO existe"}, default=str)
        }
    query = f"select * from {ENV}.orden_venta where estado = '{get_estado}';"
    pedido = run_query(query)
    return {
        "statusCode": 200,
        "body": json.dumps({"data": pedido}, default = str)
    }
        
print(get_orden_venta_by_estado(input_payload2, None))


# get productos (select id, nombre from producto)

In [ ]:
get_productos_query = f'SELECT id,nombre FROM {ENV}.producto'

# Obtengo todos los productos
productos = run_query(get_productos_query)
print(productos)

# get ordenes_produccion limit 20

In [ ]:
get_ordenes_produccion_limit20_query = f'SELECT * FROM {ENV}.orden_produccion ORDER BY fecha_creacion DESC LIMIT 20'

# Obtengo todos las ultimas 20 ordenes de produccion
ordenes_produccion = run_query(get_ordenes_produccion_limit20_query)
print(ordenes_produccion)

# post cambiar estado orden produccion

# post datos login empleado

In [ ]:
ejemplo_input_payload_correcto = {
    "email": "maria.rodriguez@frozen.com",
    "password": "contramaria", 
    "rol" : "supervisor"
}
ejemplo_input_payload_incorrecto = {
    "email": "raul@gmail.com",
    "password": "12345", 
    "rol" : "supervisor"
}
ejemplo_incompleto = {
    "email": "raul@gmail.com",
    "password": "12345"
}
ejemplo_vacio = {}


def post_login_empleado(event, context):
    
    input_payload = event
    
    required = [
        "email",
        "password",
        "rol"
    ]
    missing = [key for key in required if not input_payload.get(key)]
    if missing:
        return {
            "statusCode": 400,
            "body": json.dumps({"message": "Faltan campos obligatorios"})
        }  
    
    email = input_payload.get("email")
    password = input_payload.get("password")
    rol = input_payload.get("rol")
    
    query_rol = f"SELECT id, nombre FROM {ENV}.rol where nombre = 'supervisor'"
    rol_result = run_query(query_rol)
    if not rol_result:
        return {
            "statusCode": 400,
            "body": json.dumps({"message": f"Rol '{rol}' no encontrado"})
        } 
    id_rol = rol_result[0]["id"]
       
    query_empleado = f"SELECT id FROM {ENV}.empleado WHERE email = '{email}' AND id_rol = {id_rol}"
    empleado_result = run_query(query_empleado)
    if not empleado_result:
        query_email = f"SELECT 1 FROM {ENV}.empleado WHERE email = '{email}'"
        if not run_query(query_email):
            return {
                "statusCode": 400,
                "body": json.dumps({"message": "Email no encontrado"})
            }
        else:
            return {
                "statusCode": 400,
                "body": json.dumps({"message": "El email no corresponde con el rol indicado"})
            }
    id_empleado = empleado_result[0]["id"]
    
    query_password = f"SELECT 1 FROM {ENV}.sesion WHERE id_empleado = {id_empleado} AND password = '{password}'"
    if run_query(query_password):
        return {
            "statusCode": 200,
            "body": json.dumps({"message": "Inicio de sesion verificado"})
        }
    return {
        "statusCode": 400,
        "body": json.dumps({"message": "Contraseña incorrecta"})
    }
    
print(post_login_empleado(ejemplo_input_payload_correcto, None))

# post pedido ventas

# post cambiar estado pedido venta